<a href="https://colab.research.google.com/github/mechhector/leaf_recognition_paper_ICPR2022/blob/main/shape_textural_fe_leaf_recognition_paper_ICPR22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Functions

In [ ]:
!git remote add origin git@github.com:mechhector/visao_computacional_20212.git

fatal: not a git repository (or any of the parent directories): .git


In [ ]:
# HU Moments

def HU_FE(image):
        moments = cv2.moments(image.astype(np.float64))
        return np.asarray(cv2.HuMoments(moments).flatten())


# Local Binary Pattern

def LBP_FE(image):
        lbp_image = local_binary_pattern(image, 256, 1, "uniform")
        return np.histogram(lbp_image.ravel(), bins=256)
        #return lbp_image


# Gray Level Cooccurrency Matrix

def GLCM_FE(image):
        glcm = greycomatrix(image, [1], [0], 256, symmetric=True, normed=True)
        xs = []
        xs.append(greycoprops(glcm, 'dissimilarity')[0, 0])
        xs.append(greycoprops(glcm, 'correlation')[0, 0])
        xs.append(greycoprops(glcm, 'homogeneity')[0, 0])
        xs.append(greycoprops(glcm, 'ASM')[0, 0])
        xs.append(greycoprops(glcm, 'energy')[0, 0])
        return xs

# Returns the image's histogram

def img_histogram(merged_image, mask_image):

  '''
  Returns the image histogram
  '''

  histogram = np.zeros(256)


  for i in range(merged_image.shape[0]):
    for j in range(merged_image.shape[1]):
      
      if mask_image[i][j] == 255:

        intensity_value = merged_image[i][j]
      
        histogram[intensity_value] = histogram[intensity_value] + 1

   
  return histogram 

# Normalized Histogram

def normalize_histogram(histogram):

  norm_hist = histogram.copy()

  for i in range(histogram.shape[0]):

    norm_hist[i] = histogram[i] / histogram.sum()

  return norm_hist


# =================================================================== SHAPE FEATURES EXTRACTOR =================================================================

def getShapeFeatures(image):

  '''
  Returns a list containing shape features of an image as follows:

  [class, specimen number, eccentricity, aspect ratio, elongation, solidity, isoperimetric factor, convexity]

  '''

  shape_features_list  = []

  blur_bg_leaf_image = cv2.blur(image,(5,5))

  contours,hierarchy = cv2.findContours(blur_bg_leaf_image, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

  cnt = contours[0]
  
  if cnt.shape[0] < 5:

    blur_bg_leaf_image = cv2.blur(th_swedish_image,(7,7))

    #cv2_imshow(blur_bg_leaf_image)

    contours,hierarchy = cv2.findContours(blur_bg_leaf_image, 1, 2)

    cnt = contours[0]

  if cnt.shape[0] < 100:
    print("Not enough points.", "n =", len(cnt))
    
    return None
    
  print("Number of cnt points:" , len(cnt))

  #  SHAPE FEATURE #1 - ECCENTRICITY 

  # ellipse
  ellipse = cv2.fitEllipse(cnt)

  center, axes, orientation = ellipse

  # length of MAJOR and minor axis
  major_axis_length = max(axes)
  minor_axis_length = min(axes)

  eccentricity = np.sqrt(1-(minor_axis_length/major_axis_length)**2)

  shape_features_list.append(eccentricity)


  #  SHAPE FEATURE #2 - ASPECT RATIO

  # bounding box
  bounding_box = cv2.boundingRect(cnt)
  bx,by,bw,bh = bounding_box

  # aspect ratio
  aspect_ratio = bh/float(bw)

  shape_features_list.append(aspect_ratio)

  #  SHAPE FEATURE #3 - ELONGAITON 


  dist_transform = cv2.distanceTransform(image,cv2.DIST_L2,5)

  max_inscribed_radius = dist_transform.max()


  (x,y),radius = cv2.minEnclosingCircle(cnt)

  min_circunscribed_radius = int(radius)

  elongation = 1 - (max_inscribed_radius/min_circunscribed_radius)

  shape_features_list.append(elongation)


  #  SHAPE FEATURE #4 - SOLIDITY 

  area = cv2.contourArea(cnt)
  hull = cv2.convexHull(cnt)
  hull_area = cv2.contourArea(hull)

  solidity = float(area)/hull_area

  shape_features_list.append(solidity)

  #  SHAPE FEATURE #5 - ISOPERIMETRIC FACTOR 

  cnt_area = cv2.contourArea(cnt)
  cnt_perimeter = cv2.arcLength(cnt,True)

  isoperimetric_factor = (4 * m.pi * cnt_area) / cnt_perimeter**2

  shape_features_list.append(isoperimetric_factor)

  #  SHAPE FEATURE #6 - CONVEXITY 

  hull_perimeter = cv2.arcLength(hull,True)

  convexity = hull_perimeter / cnt_perimeter

  shape_features_list.append(convexity)

  return shape_features_list

# ==================================================================== TEXTURE FEATURES ========================================================================


def getTextureFeatures(image,image_mask):

  texture_features_list = []

  # Histogram of the merged image

  hist = cv2.calcHist([image], [0], image_mask, [256], [0, 256])

  histogram = hist.reshape((256))

  # TEXTURE FEATURE #1 - INTENSITY MEAN

  intensity_mean = image.mean()

  texture_features_list.append(intensity_mean)

  # TEXTURE FEATURE #2 - STANDART DEVIATION

  variance = moment(histogram, moment=2) / 255**2

  std_deviation = np.sqrt(variance)

  texture_features_list.append(std_deviation)

  # TEXTURE FEATURE #3 - SMOOTHNESS (R)

  smoothness_R = 1 - 1 / (1+variance)

  texture_features_list.append(smoothness_R)

  # TEXTURE FEATURE #4 - THIRD MOMENT

  third_moment = moment(histogram, moment=3) / 255**2

  texture_features_list.append(third_moment)

  # TEXTURE FEATURE # 5 - ENTROPY

  entropy_feature = entropy(histogram, base=2)

  texture_features_list.append(entropy_feature)

  return texture_features_list




### Imports and Data

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math as m
import os 


from scipy import ndimage
from google.colab.patches import cv2_imshow
from skimage.feature import greycomatrix, greycoprops
from skimage.feature import local_binary_pattern
from scipy.stats import uniform, moment, skew, entropy

dataset = 'MEW2012'

In [ ]:
img_th = cv2.imread('/content/drive/MyDrive/leaf_classification_paper/data/LEAF_dataset/LEAF_dataset_th/iPAD2_C02_EX01_B.TIFF',0)

image = cv2.imread('/content/drive/MyDrive/leaf_classification_paper/data/LEAF_dataset/LEAF_dataset/iPAD2_C02_EX01.JPG',0)

img_mg = cv2.imread('/content/drive/MyDrive/leaf_classification_paper/data/LEAF_dataset/LEAF_dataset_merged/merged_leaf_C02_EX01.JPG',0)

### Tests

In [ ]:
# Lists of threshold and merged images

th_dir_path = '/content/drive/MyDrive/leaf_classification_paper/data/MEW2012_dataset/MEW2012_t' 
mg_dir_path = '/content/drive/MyDrive/leaf_classification_paper/data/MEW2012_dataset/MEW2012_merged_images'

swedish_th_list = sorted(os.listdir(th_dir_path))

swedish_mg_list = sorted(os.listdir(mg_dir_path))

print(len(swedish_th_list), len(swedish_mg_list))

print("SW Th list:" , swedish_th_list)
print("SW Mg list:" , swedish_mg_list)

9745 9745
SW Th list: ['th_Acer_campestre_01.png', 'th_Acer_campestre_02.png', 'th_Acer_campestre_03.png', 'th_Acer_campestre_04.png', 'th_Acer_campestre_05.png', 'th_Acer_campestre_06.png', 'th_Acer_campestre_07.png', 'th_Acer_campestre_08.png', 'th_Acer_campestre_09.png', 'th_Acer_campestre_10.png', 'th_Acer_campestre_11.png', 'th_Acer_campestre_12.png', 'th_Acer_campestre_13.png', 'th_Acer_campestre_14.png', 'th_Acer_campestre_15.png', 'th_Acer_campestre_16.png', 'th_Acer_campestre_17.png', 'th_Acer_campestre_18.png', 'th_Acer_campestre_19.png', 'th_Acer_campestre_20.png', 'th_Acer_campestre_21.png', 'th_Acer_campestre_22.png', 'th_Acer_campestre_23.png', 'th_Acer_campestre_24.png', 'th_Acer_campestre_25.png', 'th_Acer_campestre_26.png', 'th_Acer_campestre_27.png', 'th_Acer_campestre_28.png', 'th_Acer_campestre_29.png', 'th_Acer_campestre_30.png', 'th_Acer_campestre_31.png', 'th_Acer_campestre_32.png', 'th_Acer_campestre_33.png', 'th_Acer_campestre_34.png', 'th_Acer_campestre_35.png

In [ ]:
# rename LEAF dataset files

# for file_name in swedish_mg_list:

  c = file_name[13:15]
  n = file_name[18:20]

  os.rename('/content/drive/MyDrive/leaf_classification_paper/data/LEAF_dataset/LEAF_dataset_merged/{}'.format(file_name), 
            '/content/drive/MyDrive/leaf_classification_paper/data/LEAF_dataset/LEAF_dataset_merged//iPAD2_C{}_EX{}_MG.JPG'.format(c,n))


In [ ]:
print(len(swedish_th_list[0]),swedish_th_list[0])

print(len(swedish_th_list[220]), swedish_th_list[220])

### Feature Extraction

In [ ]:
# MEW 2012 CLASS AND SPECIMEN NUMBER

mg_mew2012_list = sorted(os.listdir('/content/drive/MyDrive/leaf_classification_paper/data/MEW2012_dataset/MEW2012_merged_images'))


print(mg_mew2012_list)

c = 0

def getMEW2012ClassIndex(class_list):
  class_sn_list = []

  leaf_name = image_name
  print(leaf_name)
  underline = '_'
  underline_positions = [pos for pos, char in enumerate(leaf_name) if char == underline]

  print(underline_positions)

  mew2012_class = leaf_name[underline_positions[0]+1:underline_positions[2]]

  


def getMEW2012ClassAndSN(image_name):
  class_sn_list = []


  leaf_name = image_name
  
  underline = '_'
  underline_positions = [pos for pos, char in enumerate(leaf_name) if char == underline]

  #print(image_name)
  leaf_class = leaf_name[underline_positions[0]+1:underline_positions[2]]
  class_specimen_number = leaf_name[underline_positions[2]+1:underline_positions[2]+3:]

  class_sn_list.append(leaf_class)
  class_sn_list.append(int(class_specimen_number))

  return(class_sn_list)
  

mew2012_label_list = []

mew2012_specimen_number_list = []

class_of_i =  []

for i in  mg_mew2012_list:



  mew2012_classAndSpecimen = getMEW2012ClassAndSN(i) # To get a list [class, specimen number]

  mew2012_label_list.append(mew2012_classAndSpecimen[0])
  mew2012_specimen_number_list.append(mew2012_classAndSpecimen[1])


  #print(mew2012_classAndSpecimen)

  '''
  c = c+1
  if c>4:
    break
  '''




# To get the class list as a number one
class_number = 1
cl_label = 'Acer_campestre'
class_label_as_number_list = []

for h in mew2012_label_list:
  if h == cl_label:
    class_label_as_number_list.append(class_number)
  else:
    class_number = class_number + 1
    cl_label = h
    class_label_as_number_list.append(class_number)



print(mew2012_label_list)

print("Class:", class_label_as_number_list)

print("Specimen number:", mew2012_specimen_number_list)

print(len(class_label_as_number_list),len(mew2012_label_list), len(mew2012_specimen_number_list))

['mg_Acer_campestre_01.png', 'mg_Acer_campestre_02.png', 'mg_Acer_campestre_03.png', 'mg_Acer_campestre_04.png', 'mg_Acer_campestre_05.png', 'mg_Acer_campestre_06.png', 'mg_Acer_campestre_07.png', 'mg_Acer_campestre_08.png', 'mg_Acer_campestre_09.png', 'mg_Acer_campestre_10.png', 'mg_Acer_campestre_11.png', 'mg_Acer_campestre_12.png', 'mg_Acer_campestre_13.png', 'mg_Acer_campestre_14.png', 'mg_Acer_campestre_15.png', 'mg_Acer_campestre_16.png', 'mg_Acer_campestre_17.png', 'mg_Acer_campestre_18.png', 'mg_Acer_campestre_19.png', 'mg_Acer_campestre_20.png', 'mg_Acer_campestre_21.png', 'mg_Acer_campestre_22.png', 'mg_Acer_campestre_23.png', 'mg_Acer_campestre_24.png', 'mg_Acer_campestre_25.png', 'mg_Acer_campestre_26.png', 'mg_Acer_campestre_27.png', 'mg_Acer_campestre_28.png', 'mg_Acer_campestre_29.png', 'mg_Acer_campestre_30.png', 'mg_Acer_campestre_31.png', 'mg_Acer_campestre_32.png', 'mg_Acer_campestre_33.png', 'mg_Acer_campestre_34.png', 'mg_Acer_campestre_35.png', 'mg_Acer_campestre_

#### Shape Features

In [ ]:
SWEDISH_shape_features = np.zeros((len(swedish_th_list),8))

count = 0

missing_count = 0

missing_items_list = []

shape_list = []

texture_list = []

glcm_list = []


# SHAPE FE
    
for image_name in swedish_th_list:

  th_swedish_image = cv2.imread('/content/drive/MyDrive/leaf_classification_paper/data/MEW2012_dataset/MEW2012_t/{}'.format(image_name),0)
  
  shape_features_list = []

  # To add class and specimen number into the list
  '''
  if len(image_name) == 21:
    shape_features_list.append(int(image_name[7:9]))
    shape_features_list.append(int(image_name[12:14]))       
  else:
    shape_features_list.append(int(image_name[1:2]))
    shape_features_list.append(int(image_name[4:7]))
  '''
  # Get MEW2021 class and specimen number

  mew2012_class, mew2012_class_sn =  class_label_as_number_list[swedish_th_list.index(image_name)], mew2012_specimen_number_list[swedish_th_list.index(image_name)]

  shape_features_list.append(mew2012_class)
  shape_features_list.append(mew2012_class_sn)

  #print(shape_features_list)

  


   
  shape_features = getShapeFeatures(th_swedish_image) # Shape feature extraction

  if type(shape_features) == list: # Add indexes and shape features

    specimen_and_features = shape_features_list + shape_features

    shape_list.append(specimen_and_features)

  else:    
    
    missing_count = missing_count + 1
    missing_items_list.append(image_name)
    print("Missing:", missing_count, "   ", image_name)
    class_list_after_shapefe_indexes.append(swedish_th_list.index(image_name))
    
    


  count = count + 1
  

A saída de streaming foi truncada nas últimas 5000 linhas.
Number of cnt points: 679
Number of cnt points: 671
Number of cnt points: 837
Number of cnt points: 843
Number of cnt points: 871
Number of cnt points: 895
Number of cnt points: 1108
Number of cnt points: 965
Number of cnt points: 931
Number of cnt points: 886
Number of cnt points: 977
Number of cnt points: 821
Number of cnt points: 830
Number of cnt points: 944
Number of cnt points: 1126
Number of cnt points: 889
Number of cnt points: 882
Number of cnt points: 831
Number of cnt points: 1095
Number of cnt points: 973
Not enough points. n = 21
Missing: 1429     th_Mahonia_aquifolium_31.png
Number of cnt points: 854
Number of cnt points: 819
Number of cnt points: 819
Number of cnt points: 649
Number of cnt points: 1040
Number of cnt points: 658
Number of cnt points: 891
Number of cnt points: 706
Number of cnt points: 851
Number of cnt points: 660
Number of cnt points: 1021
Number of cnt points: 789
Number of cnt points: 676
Numbe

In [ ]:
len(class_list_after_shapefe_indexes)

np.savetxt('class_list_after_shapefe_indexes.txt', np.asarray(class_list_after_shapefe_indexes))

In [ ]:
# To pop out the class indexes from the original list


class_list_after_shapefe = []

  



In [ ]:
print(len(missing_items_list), len(shape_list))


2253 7492


In [ ]:
# create a mg missing list from the th missing list

mg_missing_items_list = sorted(missing_items_list)

for r in mg_missing_items_list:
  mg_missing_items_list[mg_missing_items_list.index(r)] = r.replace('th','mg')


print(mg_missing_items_list)


print(len(mg_missing_items_list))

['mg_Acer_campestre_04.png', 'mg_Acer_campestre_05.png', 'mg_Acer_campestre_06.png', 'mg_Acer_campestre_07.png', 'mg_Acer_campestre_08.png', 'mg_Acer_campestre_09.png', 'mg_Acer_campestre_10.png', 'mg_Acer_campestre_11.png', 'mg_Acer_campestre_12.png', 'mg_Acer_campestre_13.png', 'mg_Acer_campestre_14.png', 'mg_Acer_campestre_15.png', 'mg_Acer_campestre_16.png', 'mg_Acer_campestre_17.png', 'mg_Acer_campestre_18.png', 'mg_Acer_campestre_19.png', 'mg_Acer_campestre_20.png', 'mg_Acer_campestre_21.png', 'mg_Acer_campestre_22.png', 'mg_Acer_campestre_23.png', 'mg_Acer_campestre_24.png', 'mg_Acer_campestre_25.png', 'mg_Acer_campestre_26.png', 'mg_Acer_campestre_27.png', 'mg_Acer_campestre_28.png', 'mg_Acer_campestre_29.png', 'mg_Acer_campestre_30.png', 'mg_Acer_campestre_31.png', 'mg_Acer_campestre_32.png', 'mg_Acer_campestre_33.png', 'mg_Acer_campestre_34.png', 'mg_Acer_campestre_35.png', 'mg_Acer_campestre_38.png', 'mg_Acer_campestre_58.png', 'mg_Acer_campestre_62.png', 'mg_Acer_ginnala_01

In [ ]:
# TEXTURE FE

lista = []

count = 0

for image_name in swedish_mg_list:

  if image_name in mg_missing_items_list: # checks the missing images
    print("Missing image:", image_name)
    count = count + 1
    print(count)

  '''
  else:


    # Texture Features

    index = swedish_mg_list.index(image_name)

    th_index = swedish_th_list[index]

    lista.append(image_name)
       
    
    th_swedish_image = cv2.imread('/content/drive/MyDrive/leaf_classification_paper/data/MEW2012_dataset/MEW2012_t/{}'.format(th_index),0)

    mg_swedish_image = cv2.imread('/content/drive/MyDrive/leaf_classification_paper/data/MEW2012_dataset/MEW2012_merged_images/{}'.format(image_name),0)

       

    texture_features_list = []

    texture_features = getTextureFeatures(mg_swedish_image, th_swedish_image)

    texture_list.append(texture_features)

    count = count + 1

    print(count)

    
    
    # GLCM Features
    
    glcm_features = GLCM_FE(mg_swedish_image)

    glcm_list.append(glcm_features)
    '''

print(len(lista))


Missing image: mg_Acer_campestre_04.png
1
Missing image: mg_Acer_campestre_05.png
2
Missing image: mg_Acer_campestre_06.png
3
Missing image: mg_Acer_campestre_07.png
4
Missing image: mg_Acer_campestre_08.png
5
Missing image: mg_Acer_campestre_09.png
6
Missing image: mg_Acer_campestre_10.png
7
Missing image: mg_Acer_campestre_11.png
8
Missing image: mg_Acer_campestre_12.png
9
Missing image: mg_Acer_campestre_13.png
10
Missing image: mg_Acer_campestre_14.png
11
Missing image: mg_Acer_campestre_15.png
12
Missing image: mg_Acer_campestre_16.png
13
Missing image: mg_Acer_campestre_17.png
14
Missing image: mg_Acer_campestre_18.png
15
Missing image: mg_Acer_campestre_19.png
16
Missing image: mg_Acer_campestre_20.png
17
Missing image: mg_Acer_campestre_21.png
18
Missing image: mg_Acer_campestre_22.png
19
Missing image: mg_Acer_campestre_23.png
20
Missing image: mg_Acer_campestre_24.png
21
Missing image: mg_Acer_campestre_25.png
22
Missing image: mg_Acer_campestre_26.png
23
Missing image: mg_Ac

In [ ]:
# Convert lists to array

SWEDISH_shape_features = np.asarray(shape_list).reshape((len(shape_list), len(shape_list[0])))

SWEDISH_texture_features = np.asarray(texture_list).reshape((len(texture_list), len(texture_list[0])))

SWEDISH_glcm_features = np.asarray(glcm_list).reshape((len(glcm_list), len(glcm_list[0])))


# Add lists

shape_texture_list = []
shape_glcm_list = []

for add_list in range(len(shape_list)):

  shape_texture_list.append(shape_list[add_list] + texture_list[add_list])

  shape_glcm_list.append(shape_list[add_list] + glcm_list[add_list])


SWEDISH_shape_texture_features = np.asarray(shape_texture_list).reshape((len(shape_texture_list)),len(shape_texture_list[0]))

SWEDISH_shape_glcm_features = np.asarray(shape_glcm_list).reshape((len(shape_glcm_list),len(shape_glcm_list[0])))

print(SWEDISH_shape_texture_features.shape)
print(SWEDISH_shape_glcm_features.shape)



# Save data as a .txt file

np.savetxt('/content/drive/MyDrive/leaf_classification_paper/data/leaf_rec_paper/{}_shape_features.txt'.format(dataset), SWEDISH_shape_features, delimiter=',', fmt = '%1.7f')
np.savetxt('/content/drive/MyDrive/leaf_classification_paper/data/leaf_rec_paper/{}_texture_features.txt'.format(dataset), SWEDISH_texture_features, delimiter=',', fmt = '%1.7f')
np.savetxt('/content/drive/MyDrive/leaf_classification_paper/data/leaf_rec_paper/{}_glcm_features.txt'.format(dataset), SWEDISH_glcm_features, delimiter=',', fmt = '%1.7f')
np.savetxt('/content/drive/MyDrive/leaf_classification_paper/data/leaf_rec_paper/{}_shape_texture_features.txt'.format(dataset), SWEDISH_shape_texture_features, delimiter=',', fmt = '%1.7f')
np.savetxt('/content/drive/MyDrive/leaf_classification_paper/data/leaf_rec_paper/{}_shape_glcm_features.txt'.format(dataset), SWEDISH_shape_glcm_features, delimiter=',', fmt = '%1.7f')

# Shape: [class, specimen number, eccentricity, aspect ratio, elongation, solidity, isoperimetric factor, convexity]

(7492, 13)
(7492, 13)
